In [1]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import BertTokenizer, VisualBertModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import numpy as np
from PIL import Image

# Load the Excel file and check for NaN values
file_path = 'multi-sent-p.xlsx'
df = pd.read_excel(file_path)
df = df.dropna(subset=['Label_Sentiment'])  # Remove rows with NaN in 'Label_Sentiment'
df['Label_Sentiment'] = df['Label_Sentiment'].astype(int)  # Ensure Label_Sentiment is integer type

class MemeDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        
        try:
            image = Image.open(img_name).convert("RGB")
        except FileNotFoundError:
            return None
        
        if self.transform:
            image = self.transform(image)
        
        text = self.dataframe.iloc[idx, 1]
        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        
        label = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.long)
        
        sample = {
            'image': image,
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'label': label
        }
        return sample

# Transformations for the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the dataset
dataset = MemeDataset(dataframe=df, img_dir='Memes/', transform=transform)

# Custom collate function to filter out None samples
def collate_fn(batch):
    batch = [sample for sample in batch if sample is not None]
    if len(batch) == 0:
        return None
    return torch.utils.data.dataloader.default_collate(batch)

# Define the VisualBERT-based Multimodal Model
class VisualBERTMultimodalModel(nn.Module):
    def __init__(self):
        super(VisualBERTMultimodalModel, self).__init__()
        self.visual_model = models.vgg16(weights='IMAGENET1K_V1')
        self.visual_model.classifier = nn.Sequential(*list(self.visual_model.classifier.children())[:-1])  # Remove last layer
        self.text_model = VisualBertModel.from_pretrained('uclanlp/visualbert-vqa-coco-pre')
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(4096 + 768, 2)

    def forward(self, input_ids, attention_mask, images):
        visual_features = self.visual_model(images)
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        combined = torch.cat((visual_features, text_outputs), dim=1)
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        return logits

# Initialize model, loss, and optimizer
model = VisualBERTMultimodalModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping parameters
patience = 2
best_loss = float('inf')
early_stop_counter = 0

# K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, test_idx) in enumerate(skf.split(dataset, df['Label_Sentiment'])):
    print(f'Fold {fold + 1}')
    
    train_subsampler = Subset(dataset, train_idx)
    test_subsampler = Subset(dataset, test_idx)
    
    train_dataloader = DataLoader(train_subsampler, batch_size=16, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_subsampler, batch_size=16, shuffle=False, collate_fn=collate_fn)
    
    # Training loop with early stopping
    model.train()
    for epoch in range(10):  
        epoch_loss = 0.0
        for batch in train_dataloader:
            if batch is None:
                continue
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], images=batch['image'])
            loss = criterion(outputs, batch['label'])
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation step for early stopping
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in test_dataloader:
                if batch is None:
                    continue
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], images=batch['image'])
                loss = criterion(outputs, batch['label'])
                val_loss += loss.item()
        
        print(f'Epoch {epoch + 1} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Check early stopping condition
        if val_loss < best_loss:
            best_loss = val_loss
            early_stop_counter = 0  
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f'Early stopping triggered at epoch {epoch + 1}')
                break  
    
    # Evaluation
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in test_dataloader:
            if batch is None:
                continue
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], images=batch['image'])
            _, preds = torch.max(outputs, 1)
            all_labels.extend(batch['label'].numpy())
            all_preds.extend(preds.numpy())

    # Calculate metrics with zero_division parameter
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, zero_division=1)
    precision = precision_score(all_labels, all_preds, zero_division=1)
    recall = recall_score(all_labels, all_preds, zero_division=1)
    conf_matrix = confusion_matrix(all_labels, all_preds, labels=[0, 1])

    fold_results.append({
        'fold': fold + 1,
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    })
    print(f'Fold {fold + 1} - Accuracy: {accuracy}, F1: {f1}, Precision: {precision}, Recall: {recall}')
    print(f'Confusion Matrix:\n{conf_matrix}')

# Average results across folds
avg_accuracy = np.mean([result['accuracy'] for result in fold_results])
avg_f1 = np.mean([result['f1'] for result in fold_results])
avg_precision = np.mean([result['precision'] for result in fold_results])
avg_recall = np.mean([result['recall'] for result in fold_results])

print(f'Average Accuracy: {avg_accuracy}')
print(f'Average F1 Score: {avg_f1}')
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')


/home/tipu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fold 1
Epoch 1 - Train Loss: 150.9235, Val Loss: 34.7071
Epoch 2 - Train Loss: 130.3281, Val Loss: 33.2906
Epoch 3 - Train Loss: 112.8913, Val Loss: 33.7347
Epoch 4 - Train Loss: 95.2865, Val Loss: 32.8418
Epoch 5 - Train Loss: 62.5561, Val Loss: 41.6555
Epoch 6 - Train Loss: 43.2273, Val Loss: 46.2091
Early stopping triggered at epoch 6
Fold 1 - Accuracy: 0.7520576131687243, F1: 0.7639569049951029, Precision: 0.7317073170731707, Recall: 0.7991803278688525
Confusion Matrix:
[[341 143]
 [ 98 390]]
Fold 2
Epoch 1 - Train Loss: 69.3600, Val Loss: 7.6327
Epoch 2 - Train Loss: 32.3812, Val Loss: 10.8398
Epoch 3 - Train Loss: 25.7432, Val Loss: 11.5291
Early stopping triggered at epoch 3
Fold 2 - Accuracy: 0.9268795056642637, F1: 0.9258098223615465, Precision: 0.9445628997867804, Recall: 0.9077868852459017
Confusion Matrix:
[[457  26]
 [ 45 443]]
Fold 3
Epoch 1 - Train Loss: 32.4038, Val Loss: 4.0358
Epoch 2 - Train Loss: 15.4274, Val Loss: 3.0560
Epoch 3 - Train Loss: 10.5119, Val Loss: 8.1

In [3]:
import pandas as pd
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from transformers import BertTokenizer
from PIL import Image
# Load the new test data
test_file_path = 'test_data.xlsx'
test_df = pd.read_excel(test_file_path)
test_df = test_df.dropna(subset=['Label_Sentiment'])  # Ensure no NaN values in 'Label_Sentiment'
test_df['Label_Sentiment'] = test_df['Label_Sentiment'].astype(int)  # Convert labels to integer if necessary

In [4]:

# Define the dataset class for testing
class MemeTestDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        
        try:
            image = Image.open(img_name).convert("RGB")
        except FileNotFoundError:
            return None
        
        if self.transform:
            image = self.transform(image)
        
        text = self.dataframe.iloc[idx, 1]
        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        
        label = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.long)
        
        sample = {
            'image': image,
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'label': label
        }
        return sample

# Define image transformations
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize the test dataset and dataloader
test_dataset = MemeTestDataset(dataframe=test_df, img_dir='Memes/', transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Evaluate the model on the new test data
model.eval()
all_test_labels = []
all_test_preds = []
with torch.no_grad():
    for batch in test_dataloader:
        if batch is None:
            continue
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], images=batch['image'])
        _, preds = torch.max(outputs, 1)
        all_test_labels.extend(batch['label'].numpy())
        all_test_preds.extend(preds.numpy())

# Calculate evaluation metrics
test_accuracy = accuracy_score(all_test_labels, all_test_preds)
test_f1 = f1_score(all_test_labels, all_test_preds, zero_division=1)
test_precision = precision_score(all_test_labels, all_test_preds, zero_division=1)
test_recall = recall_score(all_test_labels, all_test_preds, zero_division=1)
test_conf_matrix = confusion_matrix(all_test_labels, all_test_preds, labels=[0, 1])

# Print the results
print(f'Test Accuracy: {test_accuracy}')
print(f'Test F1 Score: {test_f1}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Confusion Matrix:\n{test_conf_matrix}')


Test Accuracy: 0.7888888888888889
Test F1 Score: 0.7873134328358209
Test Precision: 0.7617328519855595
Test Recall: 0.8146718146718147
Confusion Matrix:
[[215  66]
 [ 48 211]]
